In [42]:
import numpy as np
import pickle
import open3d as o3d
import matplotlib.pyplot as plt
import copy
from collections import Counter

In [43]:
# Load data
data_raw = []
tfs1 = []
tfs2 = []

for i in range(3):
    with open(f"{i + 1}.pickle", "rb") as file:
        data_raw.append(pickle.load(file, encoding='latin1'))
    with open(f"{i + 1}_tf1.pickle", "rb") as file:
        tfs1.append(pickle.load(file, encoding='latin1'))
    with open(f"{i + 1}_tf2.pickle", "rb") as file:
        tfs2.append(pickle.load(file, encoding='latin1'))

print(type(tfs1[0])) # Tf matrix 4x4
print(type(data_raw[0])) #numpy array 
print(tfs1[0].shape)
print(data_raw[0].shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(4, 4)
(480, 640)


In [44]:
# Convert data to np arr

def remove_nans(data):
    data = data[np.logical_not(np.isnan(data))]
    return data.reshape((len(data), 1))


def convert_raw_data(data):
    shape = (data['x'].shape[0] * data['x'].shape[1], 1)
    x = data['x'].astype(np.float64)
    y = data['y'].astype(np.float64)
    z = data['z'].astype(np.float64)
    r = data['r'].astype(np.float64)
    g = data['g'].astype(np.float64)
    b = data['b'].astype(np.float64)

    x = x.reshape(shape)
    y = y.reshape(shape)
    z = z.reshape(shape)
    r = r.reshape(shape)
    g = g.reshape(shape)
    b = b.reshape(shape)

    nan_arr = np.logical_not(np.isnan(x))
    x = remove_nans(x)
    y = remove_nans(y)
    z = remove_nans(z)
    shape = (len(x), 1)

    r = r[nan_arr].reshape(shape) / 255
    g = g[nan_arr].reshape(shape) / 255
    b = b[nan_arr].reshape(shape) / 255

    return np.hstack((x, y, z)), np.hstack((r, g, b))



converted_pcs = []
converted_rgbs = []
for raw in data_raw:
    converted_pc, converted_rgb = convert_raw_data(raw)
    converted_pcs.append(converted_pc)
    converted_rgbs.append(converted_rgb)

print(converted_pcs[0].shape) # n_samples x 3


(253367, 3)


In [45]:
# Converto to O3D PointCloud
pcs = []

for pts, rgb in zip(converted_pcs, converted_rgbs):
    pc = o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(pts)
    pc.colors = o3d.utility.Vector3dVector(rgb)
    pcs.append(pc)

In [46]:
# Transform pc to world frame

transformed_pcs = []
for i, pc in enumerate(pcs):
    transformed_pc = copy.deepcopy(pc).transform(tfs2[i])
    transformed_pcs.append(transformed_pc)

pc = transformed_pcs[2]
#o3d.visualization.draw_geometries([pc])

In [47]:
# Fit plane
import time

start = time.time()
plane_model, inliers = pc.segment_plane(distance_threshold=0.0075,
                                         ransac_n=3,
                                         num_iterations=1000)
print(time.time()-start)
[a, b, c, d] = plane_model
plane = np.array([a, b, c, d])
print(f"Plane equation: {a:.5f}x + {b:.5f}y + {c:.5f}z + {d:.5f} = 0")

inlier_cloud = pc.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pc.select_by_index(inliers, invert=True)
#o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

0.39044833183288574
Plane equation: -0.00317x + -0.00297y + 0.99999z + -0.74065 = 0


In [48]:
OFFSET = 0.01

def remove_points_below_plane(pc, plane, offset):
    points = np.array(pc.points)
    rgb = np.array(pc.colors)
    plane[3] -= offset
    good_map = np.dot(np.hstack((points, np.ones((points.shape[0], 1)))), plane) > 0

    pc.points = o3d.utility.Vector3dVector(points[good_map])
    pc.colors = o3d.utility.Vector3dVector(rgb[good_map])
    return pc

pc = remove_points_below_plane(pc, plane, OFFSET)
#o3d.visualization.draw_geometries([pc])

In [49]:
# Remove points far from inlier_cloud
def get_table_bbox(inliers, plane, ROIheight, offset):
    # DBScan elements above table in order to find max cluster which is table plane
    inliers = inliers.voxel_down_sample(voxel_size=0.015)
    labels = np.array(inliers.cluster_dbscan(eps=0.1, min_points=100, print_progress=True))
    max_label = labels.max()
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    inliers.colors = o3d.utility.Vector3dVector(colors[:, :3])

    counter = Counter(labels)
    counts = counter.most_common(2)
    if len(counts) == 0:
        return None
    elif len(counts) == 1:
        if counts[0][0] == -1:
            return None
        elif counts[0][0] != -1:
            return counts[0][0]
    elif len(counts) >= 2:
        if counts[0][0] == -1:
            max_class = counts[1][0]
        elif counts[0][0] != 1:
            max_class = counts[0][0]


    # remove points not in main cluster
    good_map = (labels == max_class)
    inliers.points = o3d.utility.Vector3dVector(np.array(inliers.points)[good_map])
    inliers.colors = o3d.utility.Vector3dVector(np.array(inliers.colors)[good_map])


    # Create bounding box above table plane
    plane_normal = np.array([plane[0], plane[1], abs(plane[2])]) # normal vector to plane pointing up
    plane_normal = plane_normal / np.sqrt(np.sum(plane_normal**2)) # normalize vector

    obox = inliers.get_oriented_bounding_box() # bounding box of table plane
    obox.color = (1, 0, 0)
    extent = obox.extent
    center = obox.center

    # Move center up and ROI height / 2
    new_center = center + plane_normal * (ROIheight / 2.0)
    new_extent = copy.deepcopy(extent)
    new_extent[np.argmin(extent)] = ROIheight
    print(center, extent)
    print(new_center, new_extent)
    obox.extent = new_extent
    obox.center = new_center
    #o3d.visualization.draw_geometries([inliers, obox])
    return obox


ROIheight = 0.5
ROIbox = get_table_bbox(inlier_cloud, plane, ROIheight, OFFSET)

[0.14665129 1.0162751  0.74381476] [1.00601911 0.75223942 0.01729453]
[0.14585997 1.01553294 0.99381241] [1.00601911 0.75223942 0.5       ]


In [50]:
# Crop only elements above table (in box roi)
pc = pc.crop(ROIbox)
#o3d.visualization.draw_geometries([pc, ROIbox])

In [75]:
# Remove outliers

def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    outlier_cloud.paint_uniform_color([1, 0, 0])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])


pc_new, ind = pc.remove_radius_outlier(nb_points=15, radius=0.01)
display_inlier_outlier(pc, ind)

In [91]:
# Cluster

labels = np.array(pc_new.cluster_dbscan(eps=0.03, min_points=40, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pc_new.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pc_new])

point cloud has 9 clusters
